In [1]:
!pip install pytorch-metric-learning -q
!pip install faiss-gpu -q

     |████████████████████████████████| 109 kB 8.1 MB/s 
     |████████████████████████████████| 85.5 MB 110 kB/s 


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models

from pytorch_metric_learning import losses, testers, miners, reducers, distances
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator


In [3]:
mm = models.resnet18()

In [4]:
mm

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x


### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()

    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(
                "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                    epoch, batch_idx, loss, mining_func.num_triplets
                )
            )


### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)


### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))


device = torch.device("cuda")

img_mean, img_std = (0.1307,), (0.3081,)
normalize = transforms.Normalize(img_mean, img_std)
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])

transform = transforms.Compose(
    [transforms.ToTensor(), normalize]
)

batch_size = 128

dataset1 = datasets.MNIST(".", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST(".", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(
    dataset1, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size)


num_classes = 128
# model = Net().to(device)

model = models.resnet18(True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0003)
num_epochs = 30


### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
mining_func = miners.TripletMarginMiner(margin=0.2, distance=distance, type_of_triplets="semihard")
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)
### pytorch-metric-learning stuff ###

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [6]:
model.conv1

Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

## Train the model

In [7]:
for epoch in range(1, num_epochs + 1):
    train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
    test(dataset1, dataset2, model, accuracy_calculator)

Epoch 1 Iteration 0: Loss = 0.11288091540336609, Number of mined triplets = 77185
Epoch 1 Iteration 100: Loss = 0.09515073150396347, Number of mined triplets = 3576
Epoch 1 Iteration 200: Loss = 0.09260543435811996, Number of mined triplets = 1596
Epoch 1 Iteration 300: Loss = 0.08074184507131577, Number of mined triplets = 383
Epoch 1 Iteration 400: Loss = 0.09650678932666779, Number of mined triplets = 1014


100%|██████████| 313/313 [00:03<00:00, 92.54it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.9770000000000001
Epoch 2 Iteration 0: Loss = 0.07411324977874756, Number of mined triplets = 829
Epoch 2 Iteration 100: Loss = 0.08873023092746735, Number of mined triplets = 1214
Epoch 2 Iteration 200: Loss = 0.07850538194179535, Number of mined triplets = 365
Epoch 2 Iteration 300: Loss = 0.09126027673482895, Number of mined triplets = 3243
Epoch 2 Iteration 400: Loss = 0.07482392340898514, Number of mined triplets = 1978


100%|██████████| 313/313 [00:03<00:00, 90.97it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.9835
Epoch 3 Iteration 0: Loss = 0.08733764290809631, Number of mined triplets = 152
Epoch 3 Iteration 100: Loss = 0.08650051057338715, Number of mined triplets = 1448
Epoch 3 Iteration 200: Loss = 0.08928194642066956, Number of mined triplets = 189
Epoch 3 Iteration 300: Loss = 0.09491296112537384, Number of mined triplets = 1177
Epoch 3 Iteration 400: Loss = 0.09208017587661743, Number of mined triplets = 544


...

Computing accuracy
Test set accuracy (Precision@1) = 0.9933000000000001
Epoch 29 Iteration 0: Loss = 0.0, Number of mined triplets = 0
Epoch 29 Iteration 100: Loss = 0.08871142566204071, Number of mined triplets = 1806
Epoch 29 Iteration 200: Loss = 0.01896250993013382, Number of mined triplets = 104
Epoch 29 Iteration 300: Loss = 0.0, Number of mined triplets = 0
Epoch 29 Iteration 400: Loss = 0.046012721955776215, Number of mined triplets = 205


100%|██████████| 313/313 [00:03<00:00, 94.19it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.9922000000000001
Epoch 30 Iteration 0: Loss = 0.0, Number of mined triplets = 0
Epoch 30 Iteration 100: Loss = 0.11411440372467041, Number of mined triplets = 1946
Epoch 30 Iteration 200: Loss = 0.0, Number of mined triplets = 0
Epoch 30 Iteration 300: Loss = 0.0, Number of mined triplets = 0
Epoch 30 Iteration 400: Loss = 0.08093579113483429, Number of mined triplets = 849


100%|██████████| 313/313 [00:03<00:00, 92.70it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.9931000000000001
